# Instructions

### Particle tracking with Python
Coded by Alon Grossman 2021
Revised by Gilad Pollack 2021

This code is meant to provide a relatively painless interface for particle tracking needed for the Brownian motion experiment in
the second year physics lab experiment.
The notebook is made with the intent that students will not be required to type any line of code during their work

Please read the instructions, even if the font is small.
The first thing needed to make this code work is to install the necessary python packages for trackpy as described here: http://soft-matter.github.io/trackpy/dev/installation.html

Some cells do not produce output. Others are quite long and scary. But you shouldn't interact with the code anyway, so don't worry about it. Just run them and move to the next ones.

For this demonstration, I took the first 201 frames from the movie alon9 (now in theaters). You can find it compressed in my folder in the group drive or in the following link: https://drive.google.com/file/d/1tfrVsuDJRvzOUj6XGL3xc-5ndLTgDQRk/view?usp=sharing .

#### Interacting with the notebook:
Two modes: editing (green cell edges), and command mode (cyan cell edges). In this representation, you should mostly work with command mode, which allows easy navigation with the arrow keys.

Press esc in edit mode to switch to command mode (or click the cell left edge). Press enter in command mode to enter edit mode (or click the code itself).

To run a cell, press ctrl+enter in *command* mode. you can press shift+enter to run a cell and automatically move to the next one. You can also press the "play" button to the left of each cell.

**warning**: do not try to code in command mode. In command mode, most keys function as keyboard shortcuts, some of them irreversible (such as the dreaded X - break cell (and a programmers heart)).

# Setup 

### importing:
import necessary packages into the kernel: 
    (Run the following cells using Ctrl + Enter and don't think about it. If errors appear, you may need to redo the "installing trackpy" phase)

In [ ]:
#imports
from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3
from ipywidgets import widgets

import matplotlib as mpl
import matplotlib.pyplot as plt

# change the following to %matplotlib notebook for interactive plotting
%matplotlib inline 

# Optionally, tweak styles.
mpl.rc('figure',  figsize=(10, 5))
mpl.rc('image', cmap='gray')

import numpy as np      #data analysis
import pandas as pd      #arrays
from pandas import DataFrame, Series      # for convenience
import time 

#import pims      #opening images and data files
import trackpy as tp      #the actual tracking
#import seaborn as sb     #pretty plots 
#import cv2

#### Setting widgets:
**Below are 4** long, output-less **cells**.They contain mostly variable declarations and cosmetic support. Just **run them and move forward.**

In [ ]:
#Sample info
w_date = widgets.DatePicker(disabled=False)
w_num_of_frames = widgets.BoundedIntText(value=0, min=0, step=1)
w_particle_size = widgets.BoundedFloatText(value=1.5, min=0, step=0.5)
w_solvent = widgets.Combobox(
    placeholder='Choose Solvent',
    options=['Water', 'DMSO', 'George'],
    #description='Solvent:',
    ensure_option=False,
    disabled=False)
w_protocol = widgets.FileUpload(
    accept=
    '',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)
w_descript = widgets.Textarea(
    value='',
    placeholder='Insert additional information about your sample',
    layout=widgets.Layout(height="auto", width="auto"))

info = widgets.VBox([
    widgets.Label("Date of recording:"), w_date,
    widgets.Label("number of frames:"), w_num_of_frames,
    widgets.Label("particle size, in $\mu m$:"), w_particle_size,
    widgets.Label("solvent:"), w_solvent,
    widgets.Label("Additional inforamtion:"), w_descript,
    widgets.Label("Protocol:"), w_protocol    
    
])

In [ ]:
#Parameters for linking and MSD calculations

w_mpp = widgets.BoundedFloatText(value=0.0058, min=0, description='mpp')
w_fps = widgets.BoundedIntText(value=30, min=0, description='fps')
w_window_size = widgets.BoundedIntText(value=10, min=0, description='window size')
w_memory = widgets.BoundedIntText(value=0, min=0, description='memory')
w_mintraj = widgets.BoundedIntText(value=10,min=0, description='min. traj. length')

linking_params = widgets.VBox([w_mpp, w_fps, w_window_size, w_memory, w_mintraj])

In [ ]:
# Feature finding advanced parameters (not necessary to update every time, default is good)

w_preprocess = widgets.Checkbox(value=True,
                              description='Preprocess',
                              disabled=False,
                              indent=False)
w_characterize = widgets.Checkbox(value=True,
                                description='Characterize extras',
                                disabled=False,
                                indent=False)
w_parallelization = widgets.Checkbox(value=True,
                                   description='Parallelize feature-finding',
                                   disabled=False,
                                   indent=False)
w_quiet = widgets.Checkbox(value=False,
                         description='Quiet mid-outputs',
                         disabled=False,
                         indent=False)

advanced_linking = widgets.VBox([w_preprocess, w_characterize, w_parallelization, w_quiet])

In [ ]:
#Combining to one widget:
info_list= [info, linking_params, advanced_linking]

accordion = widgets.Accordion(info_list)
accordion.set_title(0, 'Sample information')
accordion.set_title(1, 'linking parameters')
accordion.set_title(2, 'advanced settings')

## Set Experiment Data:

**You don't actually need to write anything in here**, but it's available if you want to 
It's supposed to be a matter of convenience in documenting the details of the experiment that will be saved when you save this notebook

In [ ]:
accordion

### Video file:

#### insert the video file path in the following function:

In [ ]:
import os
from pathlib import Path
filepath = 'C:\\Users\\Owner\\Documents\\Gilad\\1.5 micron 20 fps\\2.avi'
fileDir = os.path.dirname(filepath)

filename = Path(filepath).stem

In [ ]:
import fabio
import pims
import numpy as np
import cv2 as cv
import tifffile
from skimage import io
import fabio

cap = cv.VideoCapture(filepath)
success, frame = cap.read()
gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
frame_count = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
width=int(cap.get(3))
length=int(cap.get(4))


frames = np.zeros((frame_count, length, width), 'uint8')  
i = 0;

for i in range(frame_count):
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    frames[i][:,:] = gray[0:length,0:width]
    success, frame = cap.read()

cap.release()

#### Open a frame:

In [ ]:
w_frame=widgets.BoundedIntText(value=0, min=0, max=len(frames)-1, description='frame', continuous_update=False)
emptyCenters = pd.DataFrame(columns = ["x","y"])
func=lambda i: tp.annotate(emptyCenters,frames[i])

frame_view=widgets.interact(func, i=w_frame)

# 

# Analysis

## Determine Initial Parameters

### Mass (integration over intensity) distribution in the chosen frame:
The low value columns represent the background level brightness. The minmass (which will be set later) should be set greater than these values.

### Parameters for feature finding:

#### Supporting function and widgets setup:

In [ ]:
#Supporting function
def s_annotate(s_diameter, s_minmass, s_threshold, s_markersize):
    pos = tp.locate(pims.as_grey(frames[0]),
                    diameter=s_diameter,
                    minmass=s_minmass,
                    threshold=s_threshold,
                    percentile=0,
                    separation=s_diameter + 2)
    pic = tp.annotate(pos, frames[0], plot_style={'markersize': s_markersize})
    return pic

In [ ]:
#interactive

#scaleable parameters
s_diameter = widgets.IntSlider(19, 1, 51, 2)
s_minmass = widgets.IntSlider(1000, 1, 5000, 1, continuous_update=False)
s_threshold = widgets.IntSlider(1, 1, 100, 1, continuous_update=False)
s_markersize = widgets.IntSlider(18, 0, 40, continuous_update=False)

#set user interface
ui = widgets.VBox([
    widgets.Label(value="diameter"), s_diameter,
    widgets.Label(value="minmass"), s_minmass,
    widgets.Label(value="threshold"), s_threshold,
    widgets.Label(value="markersize"), s_markersize
])

out = widgets.interactive_output(
    s_annotate, {
        's_diameter': s_diameter,
        's_minmass': s_minmass,
        's_threshold': s_threshold,
        's_markersize': s_markersize
    })



In [ ]:
checkedFrame=tp.locate(frames[w_frame.value],21)

fig, ax = plt.subplots()
ax.hist(checkedFrame['mass'], bins=20)
 
ax.set(xlabel='mass', ylabel='count') #Label the axes 

#### Setting the parameters:
*Threshold is best left at 1. minmass is easier to adjust.*

<div class="mark">
add option to choose the frame</div><i class="fa fa-lightbulb-o "></i>

In [ ]:
display(ui, out)

### Save the parameters:
Save the parameters set in the previous cell, with few (constant) others.


Please make sure that the cell above indeed displays the parameters you chose (as re-running it will erase your selection).

In [ ]:
#most important: diameter
diameter=s_diameter.value

#parameters:
params = {
    'diameter': diameter,
    'minmass': s_minmass.value,
    'maxsize': None,
    'separation': diameter+2,    #default is +1, should be +2 (according to Yael)
    'noise_size': 1,
    'smoothing_size': diameter,   # boxcar (rolling average) smoothing, in pixels. Default is diameter
    'threshold': s_threshold.value, #1/255,             #By default, 1 for integer images and 1/255 for float images.
    'invert': False,  
    'percentile': 0, #64,          #Yael recommends to leave this alone.
    'topn': None,
    'preprocess': w_preprocess.value,
    'max_iterations': 10,
    'filter_before': None,
    'filter_after': None,
    'characterize': w_characterize.value,
    'engine': 'auto'
}

if w_parallelization.value==True:
    paralel='auto'
else :
    paralel=1
    
tp.quiet(w_quiet.value)

### Check everything works:
Analyze a frame with the chosen parameters:

In [ ]:
#%%time 
w_annotate = lambda i: tp.annotate(tp.locate(frames[i], **params),
                                   frames[i],
                                   plot_style={'markersize': s_markersize.value})

widgets.interact(w_annotate, i=w_frame)


## Analyze all frames


In [ ]:
%%time
detectedParticles = tp.batch(frames,**params)

### Save/load initial particle detection results
The following two boxes allow saving to excel and loading results from an existing excel

In [ ]:
detectedParticles.to_excel(fileDir + '\\' + filename + '_detection.xlsx')

In [ ]:
detectedParticles = pd.read_excel(fileDir + '\\' + filename + '_detection.xlsx')

### Compute subpixel error:
*Of the whole movie

In [ ]:
[[ax1,ax2]] = tp.subpx_bias(detectedParticles)
ax1.set_xlabel("pixels")
ax1.set_ylabel("counts")
ax2.set_xlabel("pixels")
ax2.set_ylabel("counts")

## Calculate Trajectories
and filter short paths and/or blurry particles

### The linking parameters:
Were set earlier. Can be changed now, of course.

In [ ]:
display(w_window_size, w_memory)

### Perform linking:

<div class="mark">
generate immediate response + add  from frame to frame</div><i class="fa fa-lightbulb-o "></i>

In [ ]:
track = tp.link(detectedParticles, search_range=w_window_size.value, memory=w_memory.value)
print('A total of ', track['particle'].nunique(), 'different particles were found.')

### Save/load initial Trajectories

In [ ]:
track.to_excel(fileDir + '\\' + filename + '_track_unfiltered.xlsx')

In [ ]:
track = pd.read_excel(fileDir + '\\' + filename + '_track_unfiltered.xlsx')

### Filter short trajectories

#### Minimum trajectory length:

In [ ]:
display(w_mintraj)

#### Filter:

In [ ]:
trackShortFiltered = tp.filter_stubs(track, w_mintraj.value )  #filter out short trajectories
print('Before:', track['particle'].nunique())
print('After:', trackShortFiltered['particle'].nunique())

#### mass vs. size plot:

In [ ]:
plt.figure()
groupTable = trackShortFiltered.groupby('particle').mean()
tp.mass_size(trackShortFiltered.groupby('particle').mean()); # convenience function -- just plots size vs. mass

One might want to filter particles that are too weak, too large or not round enough, using the following code lines:

### Filter spurious particles:

In [ ]:
#widget setting
w_m_mass=widgets.BoundedIntText(value=500, min=0, max=10000, description='traj. minmass')
w_m_size=widgets.BoundedFloatText(value=6, min=0, description='traj. maxsize')
w_m_ecc=widgets.BoundedFloatText(value=0.3, min=0, step=0.1, description='traj. max eccentricity')

filtering_params = widgets.VBox([w_m_mass, w_m_size, w_m_ecc])

#### Set filtration parameters:

In [ ]:
filtering_params

In [ ]:
rowsToFilter = groupTable[((groupTable['mass'] <  w_m_mass.value) | (groupTable['size'] > w_m_size.value) | (groupTable['ecc'] > w_m_ecc.value))]
prtToFilter = rowsToFilter.index
finalTrack = trackShortFiltered.loc[~trackShortFiltered['particle'].isin(prtToFilter)]

print('Initial:', track['particle'].nunique())
print('After first filter:', trackShortFiltered['particle'].nunique())
print('After second filter:', finalTrack['particle'].nunique())


### Save/load final trajectories after filtering


In [ ]:
finalTrack.to_excel(fileDir + '\\' + filename + '_final_track.xlsx')

In [ ]:
finalTrack = pd.read_excel(fileDir + '\\' + filename + '_final_track.xlsx')

## Plot trajectories and drift

### Plot trajectories:

measure time 

In [ ]:
plt.figure()
x=tp.plot_traj(finalTrack);

#fig = x.get_figure()
#fig.savefig("{0}/traj{1}.jpeg".format(res_loc,num))

### Compute and plot Drift:
Drift substraction is possible, but recommended to be left unused.

In [ ]:
d = tp.compute_drift(finalTrack)
d.plot()
plt.show()

#tm = tp.subtract_drift(t2.copy(), d)
tm=finalTrack

## Plots!

#### Convertion ratio between microns and pixels (mpp) and the filming frame rate (fps):

In [ ]:
display(w_mpp, w_fps)

### Time Average MSD:

In [ ]:
im = tp.imsd(tm, w_mpp.value,
             w_fps.value)  

fig, ax = plt.subplots()
ax.plot(im.index, im, 'k-', alpha=0.1)  # black lines, semitransparent
ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
       xlabel='lag time $t$ (sec)')
ax.set_xscale('log')
ax.set_yscale('log')


### Time and Ensamble Average MSD:

In [ ]:
em = tp.emsd(tm, w_mpp.value,
             w_fps.value)  # microns per pixel = 100/285., frames per second = 24

fig, ax = plt.subplots()
ax.plot(em.index, em, 'o')
ax.set_xscale('log')
ax.set_yscale('log')
ax.set(ylabel=r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]',
       xlabel='lag time $t$ (sec)')
ax.set(ylim=(1e-4, 200))

### Fit to Linear Plot:
And get diffusion coefs

In [ ]:
plt.figure()
plt.ylabel(r'$\langle \Delta r^2 \rangle$ [$\mu$m$^2$]')
plt.xlabel('lag time $t$ (sec)')
ft=tp.utils.fit_powerlaw(em)  # performs linear best fit in log space, plots]
display(ft)

In [ ]:
D=ft['A']/4
D = "{:.7f}".format(float(D))
print('D:', D, 'Micrometers^2 / sec')